In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [3]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object| Assignment Method | 
|----|-----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro|weighted|
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit|weighted|
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix|random|
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro|weighted|
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit|weighted|
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix|random|
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro|weighted|
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit|weighted|
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix|random|


In [4]:
commuter_model_nowfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_nowfh_transit = CommuterModel(ipums_df=ipums_df_19.copy()) 
commuter_model_nowfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_midwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_highwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

## No WFH

In [5]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_nowfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_nowfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_transit.ipums_df['FLAG_WALK'] = commuter_model_nowfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_nowfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_nowfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

#### Transit
### Bus
commuter_model_nowfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_transit.bus_flag_binary(
                                        home_region=commuter_model_nowfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_nowfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_transit.subway_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_nowfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Zero Work From Home
commuter_model_nowfh_transit.ipums_df["FLAG_WFH"] = 0

# ### Assign transit mode hierarchically
# commuter_model_nowfh_transit.ipums_df['First_Assignment'] = commuter_model_nowfh_transit\
#             .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])
### Assign transit mode with weights -- transit heavy
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,0 #WFH
          )

commuter_model_nowfh_transit.ipums_df['First_Assignment'] = commuter_model_nowfh_transit.WeightedAssignNoOption(
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_nowfh_transit.ipums_df['TransMode'] = commuter_model_nowfh_transit.RandAssignChosenOption(
                                                                commuter_model_nowfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_nowfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_nowfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_nowfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_nowfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_micro.bus_flag_binary(
                                        home_region=commuter_model_nowfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_nowfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_micro.subway_flag_binary(
                                            home_region=commuter_model_nowfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_nowfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_nowfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )



## E-Scooter - bump up distance
commuter_model_nowfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_nowfh_micro.ipums_df['FLAG_WALK'] = commuter_model_nowfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_micro.ipums_df["FLAG_WFH"] = 0
# ### Assign transit mode hierarchically
# commuter_model_nowfh_micro.ipums_df['First_Assignment'] = commuter_model_nowfh_micro.AssignTransModesWaterfall(
#                                                     ["FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
#                                                                                 )
### Assign transit mode with weights -- micro heavy
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,0 #EBus
           ,0 #Subway
           ,0 #CommuterRail
           ,0 #Ferry
           ,15 #EScooter
           ,40 #Walk
           ,25 #EBike
           ,0 #WFH
          )

commuter_model_nowfh_micro.ipums_df['First_Assignment'] = commuter_model_nowfh_micro.WeightedAssignNoOption(
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)


commuter_model_nowfh_micro.ipums_df['TransMode'] = commuter_model_nowfh_micro.RandAssignChosenOption(
                                                                commuter_model_nowfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_mix.bus_flag_binary(
                                        home_region=commuter_model_nowfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_mix.subway_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_mix.ipums_df['FLAG_WALK'] = commuter_model_nowfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_mix.ipums_df["FLAG_WFH"] = 0

## Keep random assignment for mixed transit option
commuter_model_nowfh_mix.ipums_df['First_Assignment'] = commuter_model_nowfh_mix.RandAssignNoOption(
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WFH'])
commuter_model_nowfh_mix.ipums_df['TransMode'] = commuter_model_nowfh_mix.RandAssignChosenOption(
                                                                commuter_model_nowfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [6]:
flag_cols = [x for x in commuter_model_nowfh_micro.ipums_df if "FLAG" in x]

In [7]:
commuter_model_nowfh_micro
commuter_model_nowfh_micro.ipums_df[flag_cols].sum()

FLAG_AUTO             1614
FLAG_MOTORCYCLE        222
FLAG_TAXICAB           280
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH                 0
dtype: int64

In [8]:
commuter_model_nowfh_transit
commuter_model_nowfh_transit.ipums_df[flag_cols].sum()

FLAG_AUTO             1691
FLAG_MOTORCYCLE        175
FLAG_TAXICAB           283
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           86
FLAG_WALK             1082
FLAG_EBIKE            7150
FLAG_WFH                 0
dtype: int64

In [9]:
commuter_model_nowfh_mix
commuter_model_nowfh_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1622
FLAG_MOTORCYCLE         27
FLAG_TAXICAB           254
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           99
FLAG_WALK             1082
FLAG_EBIKE            7267
FLAG_WFH                 0
dtype: int64

## Medium WFH
commuter_model_midwfh_micro  
commuter_model_midwfh_transit  
commuter_model_midwfh_mix  

In [10]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_midwfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_transit.ipums_df['FLAG_WALK'] = commuter_model_midwfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_midwfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_midwfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_midwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_midwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_midwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_midwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Medium Work From Home
commuter_model_midwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

# ### Assign transit mode hierarchically
# commuter_model_midwfh_transit.ipums_df['First_Assignment'] = commuter_model_midwfh_transit\
#             .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_WFH","FLAG_COMMUTERRAIL","FLAG_FERRY"])
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,40 #WFH
          )

commuter_model_midwfh_transit.ipums_df['First_Assignment'] = commuter_model_midwfh_transit.WeightedAssignNoOption(
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_midwfh_transit.ipums_df['TransMode'] = commuter_model_midwfh_transit.RandAssignChosenOption(
                                                                commuter_model_midwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_midwfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_midwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_midwfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_midwfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_micro.bus_flag_binary(
                                        home_region=commuter_model_midwfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_midwfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_micro.subway_flag_binary(
                                            home_region=commuter_model_midwfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_midwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_midwfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )

## E-Scooter
commuter_model_midwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_midwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_midwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

# ### Assign transit mode hierarchically
# commuter_model_midwfh_micro.ipums_df['First_Assignment'] = commuter_model_midwfh_micro.AssignTransModesWaterfall(
#                                                     ["FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
#                                                                                 )
### Assign transit mode with weights -- micro heavy now with mid WFH
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,0 #EBus
           ,0 #Subway
           ,0 #CommuterRail
           ,0 #Ferry
           ,15 #EScooter
           ,40 #Walk
           ,25 #EBike
           ,20 #WFH
          )

commuter_model_midwfh_micro.ipums_df['First_Assignment'] = commuter_model_midwfh_micro.WeightedAssignNoOption(
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_midwfh_micro.ipums_df['TransMode'] = commuter_model_midwfh_micro.RandAssignChosenOption(
                                                                commuter_model_midwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_midwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_midwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_midwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

## Keep random assignment for mixed transit option
commuter_model_midwfh_mix.ipums_df['First_Assignment'] = commuter_model_midwfh_mix.RandAssignNoOption(
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_midwfh_mix.ipums_df['TransMode'] = commuter_model_midwfh_mix.RandAssignChosenOption(
                                                                commuter_model_midwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [11]:
commuter_model_midwfh_micro.ipums_df[flag_cols].sum() 

FLAG_AUTO             1643
FLAG_MOTORCYCLE        196
FLAG_TAXICAB           278
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              3361
dtype: int64

In [12]:
commuter_model_midwfh_transit.ipums_df[flag_cols].sum()  

FLAG_AUTO             1603
FLAG_MOTORCYCLE        167
FLAG_TAXICAB           291
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           81
FLAG_WALK             1082
FLAG_EBIKE            7206
FLAG_WFH              3263
dtype: int64

In [13]:
commuter_model_midwfh_mix.ipums_df[flag_cols].sum()  

FLAG_AUTO             1701
FLAG_MOTORCYCLE         31
FLAG_TAXICAB           290
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           88
FLAG_WALK             1082
FLAG_EBIKE            7210
FLAG_WFH              3222
dtype: int64

## High WFH
commuter_model_highwfh_micro  
commuter_model_highwfh_transit  
commuter_model_highwfh_mix  

In [14]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_highwfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_highwfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_transit.ipums_df['FLAG_WALK'] = commuter_model_highwfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_highwfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_highwfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_highwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_highwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_highwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_highwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## High Work From Home
commuter_model_highwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_highwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 1)

### Assign transit mode hierarchically
# commuter_model_highwfh_transit.ipums_df['First_Assignment'] = commuter_model_highwfh_transit\
#             .AssignTransModesWaterfall(["FLAG_WFH","FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,100 #WFH
          )

commuter_model_highwfh_transit.ipums_df['First_Assignment'] = commuter_model_highwfh_transit.WeightedAssignNoOption(
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_highwfh_transit.ipums_df['TransMode'] = commuter_model_highwfh_transit.RandAssignChosenOption(
                                                                commuter_model_highwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_highwfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_highwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_highwfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_highwfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_micro.bus_flag_binary(
                                        home_region=commuter_model_highwfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_highwfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_micro.subway_flag_binary(
                                            home_region=commuter_model_highwfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_highwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_highwfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )


## E-Scooter
commuter_model_highwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_highwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_highwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_highwfh_micro\
                                                    .wfh_flag_binary(wfh_dampener = 1)

### Assign transit mode hierarchically
# commuter_model_highwfh_micro.ipums_df['First_Assignment'] = commuter_model_highwfh_micro.AssignTransModesWaterfall(
#                                                     ["FLAG_WFH","FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
#                                                                                 )
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,0 #EBus
           ,0 #Subway
           ,0 #CommuterRail
           ,0 #Ferry
           ,15 #EScooter
           ,40 #Walk
           ,25 #EBike
           ,100 #WFH
          )

commuter_model_highwfh_micro.ipums_df['First_Assignment'] = commuter_model_highwfh_micro.WeightedAssignNoOption(
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_highwfh_micro.ipums_df['TransMode'] = commuter_model_highwfh_micro.RandAssignChosenOption(
                                                                commuter_model_highwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_highwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_highwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_highwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 1)

## Keep random assignment for mixed transit option
commuter_model_highwfh_mix.ipums_df['First_Assignment'] = commuter_model_highwfh_mix.RandAssignNoOption(
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_highwfh_mix.ipums_df['TransMode'] = commuter_model_highwfh_mix.RandAssignChosenOption(
                                                                commuter_model_highwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [15]:
commuter_model_highwfh_micro.ipums_df[flag_cols].sum()  

FLAG_AUTO             1706
FLAG_MOTORCYCLE        200
FLAG_TAXICAB           277
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              6565
dtype: int64

In [16]:
commuter_model_highwfh_transit.ipums_df[flag_cols].sum()    

FLAG_AUTO             1670
FLAG_MOTORCYCLE        210
FLAG_TAXICAB           315
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER          106
FLAG_WALK             1082
FLAG_EBIKE            7185
FLAG_WFH              6541
dtype: int64

In [17]:
commuter_model_highwfh_mix.ipums_df[flag_cols].sum()    

FLAG_AUTO             1704
FLAG_MOTORCYCLE         23
FLAG_TAXICAB           275
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           83
FLAG_WALK             1082
FLAG_EBIKE            7295
FLAG_WFH              6626
dtype: int64

In [18]:
### Compare
commuter_model_highwfh_transit.name = "commuter_model_highwfh_transit"

commuter_model_nowfh_micro.name = "commuter_model_nowfh_micro"
commuter_model_nowfh_transit.name = "commuter_model_nowfh_transit"
commuter_model_nowfh_mix.name = "commuter_model_nowfh_mix"
commuter_model_midwfh_micro.name = "commuter_model_midwfh_micro"
commuter_model_midwfh_transit.name = "commuter_model_midwfh_transit"
commuter_model_midwfh_mix.name = "commuter_model_midwfh_mix"
commuter_model_highwfh_micro.name = "commuter_model_highwfh_micro"
commuter_model_highwfh_transit.name = "commuter_model_highwfh_transit"
commuter_model_highwfh_mix.name = "commuter_model_highwfh_mix"

In [19]:
master_cts = pd.DataFrame()
col_names = list()
for model in [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix]:
    model_cts = model.ipums_df['First_Assignment'].value_counts(dropna=False)
    master_cts = pd.concat([master_cts,model_cts],axis=1)
    col_names.append(model.name)
master_cts.columns = col_names
master_cts

,commuter_model_nowfh_micro,commuter_model_nowfh_transit,commuter_model_nowfh_mix,commuter_model_midwfh_micro,commuter_model_midwfh_transit,commuter_model_midwfh_mix,commuter_model_highwfh_micro,commuter_model_highwfh_transit,commuter_model_highwfh_mix
Bicycle,10699.0,NaN,2659.0,10154.0,NaN,2479,8627.0,NaN,2470
No Option,9572.0,1668.0,1404.0,8018.0,1448.0,1240,6600.0,1256.0,1056
Escooter,3139.0,NaN,26.0,2967.0,NaN,27,2614.0,NaN,21
Walk,568.0,90.0,320.0,581.0,76.0,345,482.0,72.0,299
Subway,NaN,11907.0,8035.0,NaN,11267.0,7774,NaN,9916.0,7224
Bus,NaN,5950.0,5636.0,NaN,5649.0,5330,NaN,5045.0,5207
CommuterRail,NaN,3893.0,4174.0,NaN,3433.0,3895,NaN,2583.0,3484
Ferry,NaN,470.0,935.0,NaN,494.0,860,NaN,430.0,879
AutoOccupants,NaN,NaN,700.0,NaN,NaN,712,NaN,NaN,657
Taxicab,NaN,NaN,78.0,NaN,NaN,78,NaN,NaN,79


## Electric Model

### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object| Electric Model Object|
|----|-----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro|electric_model_nowfh_micro|
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit|electric_model_nowfh_transit|
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix|electric_model_nowfh_mix|
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro|electric_model_midwfh_micro|
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit|electric_model_midwfh_transit|
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix|electric_model_midwfh_mix|
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro|electric_model_highwfh_micro|
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit|electric_model_highwfh_transit|
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix|electric_model_highwfh_mix|

In [20]:
electric_model_nowfh_micro = ElectricModel(commuter_model_nowfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_micro')

electric_model_nowfh_transit = ElectricModel(commuter_model_nowfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_transit')

electric_model_nowfh_mix = ElectricModel(commuter_model_nowfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_mix')

electric_model_midwfh_micro = ElectricModel(commuter_model_midwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_micro')

electric_model_midwfh_transit = ElectricModel(commuter_model_midwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_transit')

electric_model_midwfh_mix = ElectricModel(commuter_model_midwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_mix')

electric_model_highwfh_micro = ElectricModel(commuter_model_highwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_micro')

electric_model_highwfh_transit = ElectricModel(commuter_model_highwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_transit')

electric_model_highwfh_mix = ElectricModel(commuter_model_highwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_mix')

In [21]:
ct= 0 
for e_m in [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix]:
    ct+=1
    
    ## Run methods in order to read in, clean, and aggregate data
    e_m.read_electric_info_in()

    e_m.clean_commuter_df()

    e_m.create_in_out_flow_dfs()

    e_m.aggregate_data()
    print(f"electric model done for em {ct}: {e_m.model_name}")

ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 1: electric_model_nowfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 2: electric_model_nowfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 3: electric_model_nowfh_mix
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 4: electric_model_midwfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 5: electric_model_midwfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 6: electric_model_midwfh_mix
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 7: electric_model_highwfh_

In [22]:
save_dirs = ["NoWFH_Micro"
,"NoWFH_Transit"
,"NoWFH_Mix"
,"MidWFH_Micro"
,"MidWFH_Transit"
,"MidWFH_Mix"
,"HighWFH_Micro"
,"HighWFH_Transit"
,"HighWFH_Mix"]

commuter_models = [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix]

electric_models = [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix]



In [23]:
save_dir = "../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2"
os.makedirs(save_dir,exist_ok=True)

for save_loc,commuter_model,electric_model in zip(save_dirs,commuter_models,electric_models):
    save_loc_models = f"{save_dir}/{save_loc}"
    print(save_loc_models)
    print(commuter_model.name," | ",electric_model.model_name)
    os.makedirs(save_loc_models,exist_ok=True)
    
    commuter_model.ipums_df.to_pickle(f"{save_loc_models}/commuter_model_ipums_df.pkl")
    
    electric_model.df_aggregate.to_pickle(f"{save_loc_models}/electric_model_df_aggregate.pkl")
    print()

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/NoWFH_Micro
commuter_model_nowfh_micro  |  electric_model_nowfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/NoWFH_Transit
commuter_model_nowfh_transit  |  electric_model_nowfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/NoWFH_Mix
commuter_model_nowfh_mix  |  electric_model_nowfh_mix

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/MidWFH_Micro
commuter_model_midwfh_micro  |  electric_model_midwfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/MidWFH_Transit
commuter_model_midwfh_transit  |  electric_model_midwfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/MidWFH_Mix
commuter_model_midwfh_mix  |  electric_model_midwfh_mix

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V2/HighWFH_Micro
commuter_model_highwfh_micro  |  electri